In [1]:
# Assignment_2: Stepwise Regression
# Student Name: Yu-Hsuan Tseng


import pandas as pd
import numpy as np
import statsmodels.api as sm

C:\Users\duck7\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
## read the data as dataframe
data = pd.read_csv('compustat_annual_2000_2017_with link information.csv')

## Clean data and impute the missing values with median
missing_percentage = data.isnull().sum()/data.shape[0]
data1 = data.loc[:, missing_percentage < 0.7]
data2 = data1._get_numeric_data()
data3 = data2.fillna(data2.median())


## Make the column "Operating Income After Depreciation" as dependent variables Y
X = data3.loc[:, data3.columns != 'oiadp']
Y = data3['oiadp']

C:\Users\duck7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2,17,33,37,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
## Stepwise procedure
def stepwise_model(X, Y):
    added = []
    while True:
        changed=False
        # forward step: add the variables which has the lowest p-value and p-value is lower than 0.05 into the list
        candidates = list(set(X.columns)-set(added))
        added_pvalue = pd.Series(index=candidates) # it has index but no value
        for column in candidates:
            model = sm.OLS(Y, sm.add_constant(pd.DataFrame(X[added+[column]]))).fit()
            added_pvalue[column] = model.pvalues[column]
        min_pvalue = added_pvalue.min()
        if min_pvalue < 0.05:
            best_candidates = added_pvalue.argmin() # Returns the indice of the minimum values
            added.append(best_candidates)
            changed=True

# backward step: step back to see if all the variables in the "added" list are all significant, if not, remove that variable
        model = sm.OLS(Y, sm.add_constant(pd.DataFrame(X[added]))).fit()
        # use all coefs except intercept
        p_values = model.pvalues.iloc[1:]
        max_pvalue = p_values.max() # null if pvalues is empty
        if max_pvalue > 0.050001:
            changed=True
            worst_candidates = p_values.argmax()
            added.remove(worst_candidates)

        if not changed:
            break

    return added

result = stepwise_model(X, Y)

print('Selected columns by doing Stepwise Regression: ', result)

C:\Users\duck7\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2,17,33,37,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Add  dxd3                           with p-value 0.0
Add  intc                           with p-value 0.0
Add  lifr                           with p-value 0.0
Add  csho                           with p-value 0.0
Add  ch                             with p-value 0.0
Add  prstkc                         with p-value 0.0
Add  sstk                           with p-value 0.0
Add  pifo                           with p-value 0.0
Add  xpp                            with p-value 0.0
Add  cibegni                        with p-value 0.0
Add  txs                            with p-value 0.0
Drop  lifr                           with p-value 0.160478
Add  spi                            with p-value 0.0
Add  pidom                          with p-value 0.0
Add  pstkl                          with p-value 0.0
Drop  pifo                           with p-value 0.892814
Add  dlcch                          with p-value 0.0
Add  txc                            with p-value 0.0
Add  ceql                         

Add  recco                          with p-value 5.01604e-08
Add  siv                            with p-value 1.27673e-07
Add  drlt                           with p-value 3.62155e-05
Add  optosey                        with p-value 0.000111368
Add  ibc                            with p-value 0.000223734
Add  xpp                            with p-value 9.83336e-06
Add  dcvsr                          with p-value 0.00028907
Add  txbcof                         with p-value 0.00117998
Add  xrd                            with p-value 0.0016318
Add  msa                            with p-value 0.00104225
Add  dvpsp_f                        with p-value 0.00274345
Add  dlrsn                          with p-value 0.00267735
Add  dxd5                           with p-value 0.00348601
Add  dd2                            with p-value 9.655e-05
Add  lifrp                          with p-value 0.00247357
Add  txditc                         with p-value 0.00220556
Add  txdc                           

In [10]:
# Discard the rest from both test and train.
new_X = X.loc[:,X.columns.isin(result)]

print(new_X.shape)

(108683, 119)


In [14]:
# run simple OSL regression model and calculate the MSE
from sklearn import metrics

model = sm.OLS(Y, new_X).fit()
Y_pred = model.predict(new_X)
MSE = metrics.mean_squared_error(Y, Y_pred)
print('MSE: ',MSE)

# Print out the statistics
model.summary()

MSE:  4.57058423252


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  oiadp   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.136e+09
Date:                Sat, 03 Nov 2018   Prob (F-statistic):               0.00
Time:                        07:49:29   Log-Likelihood:            -2.3679e+05
No. Observations:              108683   AIC:                         4.738e+05
Df Residuals:                  108565   BIC:                         4.750e+05
Df Model:                         117                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
GVKEY      -1.813e-06   1.49e-07    -12.174      0.000   -2.11e-06   -1.52e-06
LPERMCO     1.066e-05   6.65e-07     16.024      0.000    9.35e-06     1.2e-05
LINKDT     -1.084e-06   9.46e-08    -11.458      0.000   -1.27e-06   -8.99e-07
ismod          4.6340      0.072     64.082      0.000       4.492       4.776
pddur          0.5854      0.037     15.815      0.000       0.513       0.658
scf           -0.1682      0.077     -2.177      0.029      -0.320      -0.017
src           -0.0088      0.000    -20.053      0.000      -0.010      -0.008
upd           -2.3464      0.040    -59.159      0.000      -2.424      -2.269
fdate       5.191e-06      4e-07     12.975      0.000    4.41e-06    5.97e-06
pdate      -3.047e-06   4.09e-07     -7.456      0.000   -3.85e-06   -2.25e-06
acodo       2.335e-05   1.03e-05      2.264      0.024    3.13e-06    4.36e-05
act        -1.436e-05   3.47e-06     -4.139      0.000   -2.12e-05   -7.56e-06
aociother   6.962e-05   1.87e-05      3.725      0.000     3.3e-05       0.000
aocipen     3.374e-05   1.53e-05      2.209      0.027    3.81e-06    6.37e-05
aoloch      1.141e-05    3.5e-06      3.260      0.001    4.55e-06    1.83e-05
ceq        -5.001e-05   1.11e-05     -4.511      0.000   -7.17e-05   -2.83e-05
ceql        1.871e-05   6.84e-06      2.736      0.006    5.31e-06    3.21e-05
chech      -1.699e-05    4.7e-06     -3.619      0.000   -2.62e-05   -7.79e-06
cisecgl    -6.316e-05   2.43e-05     -2.594      0.009      -0.000   -1.54e-05
cogs          -0.1275      0.002    -54.617      0.000      -0.132      -0.123
cshfd          0.0003   8.96e-05      3.273      0.001       0.000       0.000
cshi          -0.0001   5.28e-05     -2.186      0.029      -0.000   -1.19e-05
csho           0.0003      0.000      2.900      0.004       0.000       0.001
cshpri        -0.0004      0.000     -2.582      0.010      -0.001    -8.7e-05
dcpstk         0.0003    4.8e-05      6.903      0.000       0.000       0.000
dcvsr         -0.0002   5.48e-05     -3.501      0.000      -0.000   -8.45e-05
dd1         5.929e-05   1.06e-05      5.599      0.000    3.85e-05       8e-05
dd2        -6.191e-05   1.16e-05     -5.358      0.000   -8.46e-05   -3.93e-05
dd4         6.913e-05   1.92e-05      3.609      0.000    3.16e-05       0.000
dlc        -2.227e-05   1.01e-05     -2.197      0.028   -4.21e-05    -2.4e-06
dlcch       9.503e-06   4.59e-06      2.072      0.038    5.14e-07    1.85e-05
dltt        5.277e-05   3.84e-06     13.758      0.000    4.53e-05    6.03e-05
dm          4.929e-05   4.09e-06     12.036      0.000    4.13e-05    5.73e-05
dp            -0.0199      0.000    -48.490      0.000      -0.021      -0.019
drlt        5.816e-05   1.61e-05      3.605      0.000    2.65e-05    8.98e-05
dt         -4.075e-05   2.76e-06    -14.783      0.000   -4.62e-05   -3.53e-05
dx